In [24]:
#!/usr/bin/env python
import json, re, sys, os, json, subprocess, time, logging, requests, paramiko, urllib3
from subprocess import call, check_output
from requests.structures import CaseInsensitiveDict
urllib3.disable_warnings()
import requests


#import the env vars and logon to vault to get the secret and then use it to run this command and write the output to the vault
token = os.getenv('APPD_SECRET')
APPD_SECRET='cdb74fe0-a19e-4432-9350-9bb6ebc1fa56'

url = "https://cisco-apipartnertraininglab.saas.appdynamics.com/auth/v1/oauth/token"
payload='grant_type=client_credentials&client_id=fsolab4%40cisco-apipartnertraininglab&client_secret=' + APPD_SECRET
print(payload)
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

#print(response.text)
print(response.json)

#write the access token to the vault


grant_type=client_credentials&client_id=fsolab4%40cisco-apipartnertraininglab&client_secret=cdb74fe0-a19e-4432-9350-9bb6ebc1fa56
<bound method Response.json of <Response [200]>>


In [16]:
token_json = response.json()
print(token_json)


{'token_type': 'bearer', 'access_token': 'eyJraWQiOiJiN2UzOTg1OS1iOTE0LTRiNzItYjU3NS0wNDExYTQ2NTU4YzUiLCJhbGciOiJSUzUxMiJ9.eyJpc3MiOiJBcHBEeW5hbWljcyIsImF1ZCI6IkFwcERfQVBJcyIsImp0aSI6IlJCY3pYUUVCR1d2ZTdrUVFoSGZ2WHciLCJzdWIiOiJmc29sYWI0IiwiaWRUeXBlIjoiQVBJX0NMSUVOVCIsImlkIjoiY2NlNmI5Y2YtMWU4NC00MTI1LWI4MWYtMDhiMDU5MTEwNTE5IiwiYWNjdElkIjoiNDEzMTk1MmItNmNhOC00NWUyLWFiYmQtNTcyMGYxZmQ2YjhjIiwidG50SWQiOiI0MTMxOTUyYi02Y2E4LTQ1ZTItYWJiZC01NzIwZjFmZDZiOGMiLCJhY2N0TmFtZSI6ImNpc2NvLWFwaXBhcnRuZXJ0cmFpbmluZ2xhYiIsInRlbmFudE5hbWUiOiJjaXNjby1hcGlwYXJ0bmVydHJhaW5pbmdsYWIiLCJmbW1UbnRJZCI6bnVsbCwiYWNjdFBlcm0iOltdLCJyb2xlSWRzIjpbXSwiaWF0IjoxNjUwNTcyODExLCJuYmYiOjE2NTA1NzI2OTEsImV4cCI6MTY1MDY1OTIxMSwidG9rZW5UeXBlIjoiQUNDRVNTIn0.Ue6ZpShwzADGR16SJiHicbNTuZ1Q6CR6FvyGFNA9qqx7g_1yGpLUPC_v3VT4O2klraurD66kjzqr_NhFzoXuv5VK8qUDhAt-lYG5q0XalAgqdfFBB3oSV1iQbmbbO0HpqSO3-uyJjeg_XGx0GKNF9Pbfz0HDi6iAIC817LG3-eyQNHqo8s4JZvjGIOpVjOaOtPvpmFlOi6MJ1O9OS0bA4nCUTZ4yJcIEVKCpVbCck_Dn_Ge7gjRtrDbBdqSe740RPaY1lapX1OkrXc2uR-gRUzmv9

In [20]:
print(type(token_json))

<class 'dict'>


In [21]:
question_token = token_json['access_token']
print(question_token)

eyJraWQiOiJiN2UzOTg1OS1iOTE0LTRiNzItYjU3NS0wNDExYTQ2NTU4YzUiLCJhbGciOiJSUzUxMiJ9.eyJpc3MiOiJBcHBEeW5hbWljcyIsImF1ZCI6IkFwcERfQVBJcyIsImp0aSI6IlJCY3pYUUVCR1d2ZTdrUVFoSGZ2WHciLCJzdWIiOiJmc29sYWI0IiwiaWRUeXBlIjoiQVBJX0NMSUVOVCIsImlkIjoiY2NlNmI5Y2YtMWU4NC00MTI1LWI4MWYtMDhiMDU5MTEwNTE5IiwiYWNjdElkIjoiNDEzMTk1MmItNmNhOC00NWUyLWFiYmQtNTcyMGYxZmQ2YjhjIiwidG50SWQiOiI0MTMxOTUyYi02Y2E4LTQ1ZTItYWJiZC01NzIwZjFmZDZiOGMiLCJhY2N0TmFtZSI6ImNpc2NvLWFwaXBhcnRuZXJ0cmFpbmluZ2xhYiIsInRlbmFudE5hbWUiOiJjaXNjby1hcGlwYXJ0bmVydHJhaW5pbmdsYWIiLCJmbW1UbnRJZCI6bnVsbCwiYWNjdFBlcm0iOltdLCJyb2xlSWRzIjpbXSwiaWF0IjoxNjUwNTcyODExLCJuYmYiOjE2NTA1NzI2OTEsImV4cCI6MTY1MDY1OTIxMSwidG9rZW5UeXBlIjoiQUNDRVNTIn0.Ue6ZpShwzADGR16SJiHicbNTuZ1Q6CR6FvyGFNA9qqx7g_1yGpLUPC_v3VT4O2klraurD66kjzqr_NhFzoXuv5VK8qUDhAt-lYG5q0XalAgqdfFBB3oSV1iQbmbbO0HpqSO3-uyJjeg_XGx0GKNF9Pbfz0HDi6iAIC817LG3-eyQNHqo8s4JZvjGIOpVjOaOtPvpmFlOi6MJ1O9OS0bA4nCUTZ4yJcIEVKCpVbCck_Dn_Ge7gjRtrDbBdqSe740RPaY1lapX1OkrXc2uR-gRUzmv9Z3w2K3kqmtmzUayy-8gXpGfp4_VD5-2TL01aGtSBti

In [ ]:
agent_list = agent_list_json['agents']
list_of_dictionaries = agent_list
sought_value = "Enterprise"
found_values = []
for dictionary in list_of_dictionaries:
    if (dictionary["agentType"] == "Enterprise"):
        found_values.append(dictionary)
print(found_values)

In [22]:
#!/usr/bin/env python
import json, re, sys, os, json, subprocess, time, logging, requests, urllib3
from subprocess import call, check_output
from requests.structures import CaseInsensitiveDict
urllib3.disable_warnings()
import requests

#The AppD secret is passed in via Env Var which is injected into the build container via the pipeline
#import the env vars and logon to vault to get the secret and then use it to run this command and write the output to the vault
appd_secret = os.getenv('APPD_SECRET')
#APPD_SECRET='cdb74fe0-a19e-4432-9350-9bb6ebc1fa56'

url = "https://cisco-apipartnertraininglab.saas.appdynamics.com/auth/v1/oauth/token"
payload='grant_type=client_credentials&client_id=fsolab4%40cisco-apipartnertraininglab&client_secret=' + str(appd_secret)
print(payload)
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.json)
#put the bearer token into a var
token_json = response.json()
appd_token = token_json['access_token']
print(appd_token)

os.environ["APPD_TOKEN"] = appd_token

#write the access token to the vault (remember need to update this later so it does it for each environment - so each branch needs to have a user in vault with that name
#Do this from the shell script until you add in the vault module for python here.....



grant_type=client_credentials&client_id=fsolab4%40cisco-apipartnertraininglab&client_secret=None
<bound method Response.json of <Response [401]>>


KeyError: 'access_token'